# 不含模型参数的自定义层

自定义一个将输入减掉均值后输出的层，并将层的计算定义在forward函数里

In [55]:
import torch
from torch import nn

class CenterLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x-x.mean()

实例化此层，然后做前向计算

In [56]:
layer=CenterLayer()
layer(torch.tensor([1,2,3,4,5],dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

我们也可以用它来构造更加复杂的模型

In [57]:
net=nn.Sequential(nn.Linear(8,128),CenterLayer())

因为减去均值，所以均值接近于0

In [58]:
y = net(torch.rand(4, 8))
y.mean().item()

-3.725290298461914e-09

# 含模型参数的自定义层

自定义含模型参数的自定义层。其中模型参数可以通过训练学出。

如果一个Tensor是parameter，那么他会自动被添加到模型的参数列表。所以在自定义含模型参数的层时，我们应该将参数定义成paramter，还可以使用paramterList和paramterDict分别定义成参数的列表和字典

ParameterList接受一个Paramter实例的列表作为输入然后得到一个参数列表，在使用时可以用索引来访问某个参数，另外也可以使用append和extend在列表后面新增参数

In [59]:
class MyDense(nn.Module):
    def __init__(self):
        super().__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.rand(4,4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.rand(4,1)))

    def foprward(self,x):
        for i in range(len(self.params)):
            x=torch.mm(x,self.params[i])
        return x


In [60]:
net=MyDense()
print(net)

MyDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


ParameterDict接受一个Parameter实例的字典作为输入然后得到一个参数字典，然后就可以按照字典的规则使用

In [61]:
class MyDictDense(nn.Module):
    def __init__(self):
        super().__init__()
        self.params=nn.ParameterDict({
            'linear1':nn.Parameter(torch.randn(4,4)),
            'linear2':nn.Parameter(torch.randn(4,1))
        })
        self.params.update({'linear3':nn.Parameter(torch.rand(4,2))}) 
    def forward(self,x,choice='linear1'):
        return torch.mm(x,self.params[choice])
     

In [62]:
net=MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [63]:
x = torch.ones(1, 4)
print(net(x))
print(net(x, 'linear2'))
print(net(x, 'linear3'))


tensor([[-0.6686,  0.0424,  1.5243, -1.5850]], grad_fn=<MmBackward>)
tensor([[0.2494]], grad_fn=<MmBackward>)
tensor([[2.1037, 2.9605]], grad_fn=<MmBackward>)
